In [2]:
import paramiko
from getpass import getpass

# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password: ")

# Host information
host = "192.168.6.71"
port = 2222
username = "michael.pinelli"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationEsception:
    print("Authentication failed.")

In [6]:
command = ['sudo -S lsof -p 148658',
           'sudo -S ps -aux',
           'sudo -S cat /etc/passwd',
           'sudo -S cat /var/log/auth.log']
# Run a command
#command = ['ps -ef', 'netstat -an --inet', 'last -adx', 'cut -d: -f1 /etc/passwd']
#command = input("Which set of commands would you like to run? ")
for eachCMD in command:
    # Get the output from the command
    stdin, stdout, stderr = ssh.exec_command(eachCMD)
    
    stdin.write(thePass + "\n")

    # Get results from a stdout
    lines = stdout.readlines()

    # Convert the list to a string
    output = ''.join(lines)

    # Header output
    sepHeader = '' + '### BEGIN' + eachCMD + ' ###\n\n'

    # Footer
    sepFooter = '' + '### END' + eachCMD + ' ###\n\n'

    #Concatenate the header, output, and footer
    cmd_output = sepHeader + output + sepFooter

    # Save the cmd_output to a file
    with open('ssh_results.txt', 'a') as f:
        f.write(cmd_output)

In [ ]:
command = input("Which set of commands would you like to run? ")

# Get the output from the command
stdin, stdout, stderr = ssh.exec_command(command)
    
stdin.write(thePass + "\n")

# Get results from a stdout
lines = stdout.readlines()

# Convert the list to a string
output = ''.join(lines)

# Header output
sepHeader = '' + '### BEGIN' + command + ' ###\n\n'

# Footer
sepFooter = '' + '### END' + command + ' ###\n\n'

#Concatenate the header, output, and footer
cmd_output = sepHeader + output + sepFooter

# Save the cmd_output to a file
with open('ssh_results.txt', 'w') as f:
    f.write(cmd_output)